# Ontologias

1. [Instalación y Carga de Datos](#index-1)
2. [Creación de la Ontología](#index-2)
3. [Poblar la Ontología (Individuos)](#index-3)
4. [Consultar el Conocimiento Inferido](#index-4)
5. [Diferencias fundamentales con el ejemplo de CBR](#index-5)

* **Sistemas basados en reglas**: Estos funcionan con estructuras de tipo `SI (condición) ENTONCES (acción)`. Por ejemplo: "SI peso > 200 ENTONCES categoría = Heavyweight". Nuestro sistema no usa reglas fijas, sino que compara ejemplos (casos) para encontrar parecidos.

* **Ontologías**: Una ontología es un modelo de datos que representa conceptos y sus relaciones jerárquicas (ej. "Un Striker es un tipo de Luchador"). Aunque podríamos haber usado una para definir los términos, el motor principal de tu código se basa en la distancia/similitud entre registros, no en la navegación de una red semántica.

<a id="index-1"></a>
## **1. Instalación y Carga de Datos**

In [3]:
%pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 3.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.49-py3-none-any.whl size=23799871 sha256=b8cba8964f160c224c7175c78fde7aae6c74f011544855e5485ebfbc5bf4cd79
  Stored in directory: /home/jovyan/.cache/pip/wheels/91/52/bb/dc7de5bdab840c91a4909273950ace12b69d079ba2e3fb95d8
Successfully built owlready2
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from owlready2 import *
import re

# Cargar datos
df = pd.read_csv("casos_soporte.csv", sep=';')
df.columns = df.columns.str.strip()

# Limpieza rápida de columnas clave para la ontología
def to_float(x):
    try: return float(str(x).replace(',', '.'))
    except: return 0.0

df['TD_Avg'] = df['TD Avg'].apply(to_float) # Promedio de derribos
df['SLpM'] = df['SLpM'].apply(to_float)     # Golpes por minuto

<a id="index-2"></a>
## **2. Creación de la Ontología**
Aquí definimos el "mundo" de la UFC. Lo más potente de las ontologías es que podemos definir una Clase Definida: una clase que se llena sola si el individuo cumple ciertos requisitos.

In [5]:
# Crear ontología en memoria
onto = get_ontology("http://sbc_ufc.org/mma.owl")

with onto:
    # --- Clases Base ---
    class Atleta(Thing): pass
    class EstiloPelea(Thing): pass
    
    # --- Subclases ---
    class Striker(Atleta): pass
    class Grappler(Atleta): pass

    # --- Propiedades (Atributos) ---
    class tienePromedioDerribos(Atleta >> float, DataProperty, FunctionalProperty): pass
    class tieneGolpesPorMinuto(Atleta >> float, DataProperty, FunctionalProperty): pass

    # --- Lógica de Inferencia (SBC) ---
    # Si un luchador tiene más de 2.0 derribos por pelea, ES un Grappler
    class Grappler(Atleta):
        equivalent_to = [Atleta & tienePromedioDerribos.some(ConstrainedDatatype(float, min_inclusive = 2.0))]

    # Si da muchos golpes y derriba poco, ES un Striker
    class Striker(Atleta):
        equivalent_to = [Atleta & tieneGolpesPorMinuto.some(ConstrainedDatatype(float, min_inclusive = 4.0))]

<a id="index-3"></a>
## **3. Poblar la Ontología (Individuos)**
Ahora convertimos las filas del CSV en "Individuos" de la ontología..

In [6]:
# Tomamos una muestra para el ejemplo
for index, row in df.head(50).iterrows():
    # Creamos el individuo (usamos el nombre como ID único)
    nombre_id = re.sub(r'\W+', '', row['NAME'])
    luchador = Atleta(nombre_id)
    luchador.tienePromedioDerribos = row['TD_Avg']
    luchador.tieneGolpesPorMinuto = row['SLpM']

# Ejecutar el Razonador (Pellet o HermiT)
# Esto es lo que hace la "magia": clasifica a los luchadores según las reglas
sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /opt/conda/lib/python3.11/site-packages/owlready2/hermit:/opt/conda/lib/python3.11/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmp37p9k_s_
* Owlready2 * HermiT took 3.6797327995300293 seconds
* Owlready * Reparenting mma.AbdulKareemAlSelwady: {mma.Atleta} => {mma.Grappler, mma.Striker}
* Owlready * Reparenting mma.JuanAdams: {mma.Atleta} => {mma.Striker}
* Owlready * Reparenting mma.RicardoAbreu: {mma.Atleta} => {mma.Grappler}
* Owlready * Reparenting mma.MansurAbdulMalik: {mma.Atleta} => {mma.Striker}
* Owlready * Reparenting mma.IsraelAdesanya: {mma.Atleta} => {mma.Striker}
* Owlready * Reparenting mma.YoshihiroAkiyama: {mma.Atleta} => {mma.Grappler}
* Owlready * Reparenting mma.AshiekAjim: {mma.Atleta} => {mma.Striker}
* Owlready * Reparenting mma.OmariAkhmedov: {mma.Atleta} => {mma.Grappler}
* Owlready * Reparenting mma.HerdemAlacabek: {mma.Atleta} => {mma.Grappler}


<a id="index-4"></a>
## **4. Consultar el Conocimiento Inferido**
A diferencia del CBR, aquí no preguntamos "¿quién se parece a este?", sino "¿quién pertenece a la categoría de Grappler?".

In [7]:
print("--- RESULTADOS DEL RAZONAMIENTO ONTOLÓGICO ---")

# Listar luchadores que el sistema INFERIÓ que son Grapplers
for l in onto.Grappler.instances():
    print(f"Luchador: {l.name} | Derribos: {l.tienePromedioDerribos} -> Clasificado como: GRAPPLER")

# Listar Strikers
print("\n--- STRIKERS DETECTADOS ---")
for l in onto.Striker.instances():
    print(f"Luchador: {l.name} | Golpes: {l.tieneGolpesPorMinuto} -> Clasificado como: STRIKER")

--- RESULTADOS DEL RAZONAMIENTO ONTOLÓGICO ---
Luchador: PapyAbedi | Derribos: 3.47 -> Clasificado como: GRAPPLER
Luchador: RicardoAbreu | Derribos: 2.13 -> Clasificado como: GRAPPLER
Luchador: MohamedAdo | Derribos: 4.97 -> Clasificado como: GRAPPLER
Luchador: OmariAkhmedov | Derribos: 2.58 -> Clasificado como: GRAPPLER
Luchador: YoshihiroAkiyama | Derribos: 2.29 -> Clasificado como: GRAPPLER
Luchador: AbdulKareemAlSelwady | Derribos: 2.32 -> Clasificado como: GRAPPLER
Luchador: HerdemAlacabek | Derribos: 2.06 -> Clasificado como: GRAPPLER

--- STRIKERS DETECTADOS ---
Luchador: DarionAbbey | Golpes: 8.44 -> Clasificado como: STRIKER
Luchador: MansurAbdulMalik | Golpes: 4.27 -> Clasificado como: STRIKER
Luchador: JuanAdams | Golpes: 7.09 -> Clasificado como: STRIKER
Luchador: IsraelAdesanya | Golpes: 4.02 -> Clasificado como: STRIKER
Luchador: MariyaAgapova | Golpes: 4.43 -> Clasificado como: STRIKER
Luchador: JessicaAguilar | Golpes: 4.93 -> Clasificado como: STRIKER
Luchador: AshiekA

<a id="index-3"></a>
## **5. Diferencias fundamentales con el ejemplo de CBR**
* **Conocimiento vs. Datos**: En el CBR usábamos los datos para comparar distancias. Aquí hemos creado una regla de conocimiento: ***"Un Grappler es aquel con derribos > 2.0"***.

* **Inferencia**: En el CBR tú ordenabas los resultados. En las ontologías, el Razonador (`sync_reasoner`) es quien decide a qué clase pertenece cada luchador basándose en la lógica.

* **Jerarquía**: Podrías crear una clase EliteGrappler que sea subclase de Grappler y que además requiera un `Win_Rate > 0.8`. El sistema organizará a los luchadores en un árbol jerárquico automáticamente.